# Chapter 5 - Indoor Scene Classification with TFLu and Arduino Nano

In [ ]:
%tensorflow_version 2.x

### Python libraries

In [ ]:
import numpy as np
import pathlib
import tensorflow as tf
import tensorflow_datasets as tfds
import zipfile
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model

### Constants

In [ ]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "indoor_scene_recognition.tflite"
TFL_MODEL_HEADER_FILE = "indoor_scene_recognition_model.h"
TF_MODEL = "indoor_scene_recognition"

## Transfer learning with Keras

### Unzip the dataset (dataset.zip)

In [ ]:
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
data_dir = "dataset"

### Prepare the train (80%) and validation (20%) datasets

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

### Get the name of the classes

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

### Rescale the pixel values from [0, 255] tp [-1, 1]

In [ ]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds   = val_ds.map(lambda x, y: (rescale(x), y))

### Import the MobileNet v2 pre-trained model

In [ ]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
base_model = MobileNetV2(input_shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)
base_model.trainable = False

feat_extr = base_model

### Augment the input data

In [ ]:
augmen = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds   = val_ds.map(lambda x, y: (augmen(x), y))

### Prepare the classification head

In [ ]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

### Build the model architecture

In [ ]:
inputs = tf.keras.Input(shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3))
x = global_avg_layer(feat_extr.layers[-1].output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs, outputs=outputs)

### Compile the model with a 0.0005 learning rate

In [ ]:
lr = 0.0005
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

### Model summary

In [ ]:
model.summary()

### Train the model with 10 epochs

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

### Save the TensorFlow model

In [ ]:
model.save(TF_MODEL)

## Preparing and testing the quantized TFLite model

### Unzip the test dataset (test_samples.zip)

In [ ]:
with zipfile.ZipFile("test_samples.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
test_dir = "dataset"

### Rescale the pixel values from [0, 255] to [-1, 1]

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                      interpolation="bilinear",
                                                      image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))
test_ds  = test_ds.map(lambda x, y: (rescale(x), y))

### Quantize the TensorFlow model with the TFLite converter

In [ ]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]

converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8

tfl_model = converter.convert()

### Get the TFLite model size in bytes

In [ ]:
size_tfl_model = len(tfl_model)
print(len(tfl_model), "bytes")

### Initialize the TFLite interpreter

In [ ]:
# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_content=tfl_model)

# Allocate the tensors
interpreter.allocate_tensors()

### Get input quantization parameters

In [ ]:
# Get input/output layer information
i_details = interpreter.get_input_details()[0]
o_details = interpreter.get_output_details()[0]

# Get input quantization parameters.
i_quant = i_details["quantization_parameters"]
i_scale      = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]

### Evaluate the accuracy of the quantized TFLite model

In [ ]:
test_ds0 = val_ds.unbatch()

num_correct_samples = 0
num_total_samples   = len(list(test_ds0.batch(1)))

for i_value, o_value in test_ds0.batch(1):
  i_value = (i_value / i_scale) + i_zero_point
  i_value = tf.cast(i_value, dtype=tf.int8)
  interpreter.set_tensor(i_details["index"], i_value)
  interpreter.invoke()
  o_pred = interpreter.get_tensor(o_details["index"])[0]

  if np.argmax(o_pred) == o_value:
    num_correct_samples += 1

print("Accuracy:", num_correct_samples/num_total_samples)

### Convert the TFLite model to C-byte array with xxd

In [ ]:
open("model.tflite", "wb").write(tfl_model)
!apt-get update && apt-get -qq install xxd
!apt-get -qq install xxd
!xxd -c 60 -i model.tflite > indoor_scene_recognition.h